In [52]:
import cv2
import numpy as np

In [86]:
#https://github.com/spmallick/learnopencv/blob/5d9e05bce247e475488a40cfec0de11c006db532/Homography/utils.py

def mouse_handler(event, x, y, flags, data) :
    
    if event == cv2.EVENT_LBUTTONDOWN :
        cv2.circle(data['im'], (x,y),3, (0,0,255), 5, 16);
        cv2.imshow("Image", data['im']);
        if len(data['points']) < 4 :
            data['points'].append([x,y])

def get_four_points(im):
    
    # Set up data to send to mouse handler
    data = {}
    data['im'] = im.copy()
    data['points'] = []
    
    #Set the callback function for any mouse event
    cv2.imshow("Image",im)
    cv2.setMouseCallback("Image", mouse_handler, data)
    cv2.waitKey(0)
    
    # Convert array to np.array
    points = np.vstack(data['points']).astype(float)
    
    return points

In [87]:
fname = "./judd.webm"
cap = cv2.VideoCapture(fname)

shot = []

# TODO: Automatic table corner detection
ret, frame = cap.read()
cv2.imshow("Image", frame)
pts_src = get_four_points(frame)
size = (500, 1000, 3)
pts_dst = np.array(
               [
                [0,0],
                [size[0] - 1, 0],
                [size[0] - 1, size[1] -1],
                [0, size[1] - 1 ]
                ], dtype=float
               )

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("eof")
        break

    h, status = cv2.findHomography(pts_src, pts_dst)
    im_dst = cv2.warpPerspective(frame, h, size[:2])
    cv2.imshow("Image", im_dst)
    shot.append(im_dst)
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()


eof


In [123]:
cv2.imwrite("test.jpg", shot[0])

for frame in shot:
    # Smooth
    frame_blur = cv2.GaussianBlur(frame, (0,0), 2)
    frame_hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV)

    # Snooker green color range for judd.webm
    lowerb = np.array([52, 150, 50])
    upperb = np.array([60, 255, 220])
    mask = cv2.inRange(frame_hsv, lowerb, upperb)

    kernel = np.ones((5,5), np.uint8)
    mask_closing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel) # Cleans inner points
    
    # Applies the mask to the original frame
    _,mask_inv = cv2.threshold(mask_closing,5,255,cv2.THRESH_BINARY_INV)
    masked_img = cv2.bitwise_and(frame,frame, mask=mask_inv)

    ctrs, hierarchy = cv2.findContours(mask_inv, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    ctrs = [ctr for ctr in ctrs
                        if 300 < cv2.contourArea(ctr) < 1000
                        #and cv2.minAreaRect(ctr)[1][0]*0.28 < cv2.minAreaRect(ctr)[1][1]
                        #and cv2.minAreaRect(ctr)[1][1]*0.28 < cv2.minAreaRect(ctr)[1][0]
            ]
    frame = frame.copy()
    #draw_rectangles(ctrs, frame)
    cv2.drawContours(frame, ctrs, -1, (255, 0, 0), 3)
    cv2.imshow("FRAME", frame)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()
